# Visualizing CASP Data

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from skbio.alignment import global_pairwise_align_protein
from skbio import Protein

In [2]:
def get_tertiary_matrix(tertiary, i):
    x = tertiary[0][3 * i], tertiary[0][3 * i + 1], tertiary[0][3 * i + 2]
    y = tertiary[1][3 * i], tertiary[1][3 * i + 1], tertiary[1][3 * i + 2]
    z = tertiary[2][3 * i], tertiary[2][3 * i + 1], tertiary[2][3 * i + 2]
    return x, y, z

In [3]:
def make_training_example(lines):
    """ Takes lines read from file and created a training example. """

    example = {'id': lines[1].rstrip(),
               'primary': lines[3],
               'mask': lines[31].rstrip()}

    pssm = []
    for i in range(5, 26):
        pssm.append(lines[i])
    example['pssm'] = '\n'.join(pssm)
    
    tertiary = []
    for i in range(27, 30):
        tertiary.append(lines[i])
    example['tertiary'] = '\n'.join(tertiary)

    return example

In [4]:
def get_secondary(file_name):
    with open('data/dssp/' + file_name) as raw_dssp:
        partial_primary, secondary = [], []
        flag = False
        for line in raw_dssp:
            if flag or ('#' in line):
                if not flag:
                    flag = True
                    continue
            if flag:
                if line[7:10] == '   ':
                    continue
                p, s = line[13], line[16]
                partial_primary.append(p)
                secondary.append(s)
    partial_primary = ''.join(partial_primary)
    secondary = ''.join(secondary)
    return partial_primary, secondary

In [5]:
def align_secondary(primary, partial_primary, secondary):
    valid_aa = ['V', 'A', 'I', 'Y', 'N', 'P', 'X', 'K', '.', 'T', 'W', 'Z', 'H', '*', 'E', 'L', 'G', 'Q', '-', 'B', 'F', 'S', 'D', 'C', 'M', 'R']
    new_primary = ''
    for aa in primary:
        if aa not in valid_aa:
            new_primary += '*'
        else:
            new_primary += aa
    new_partial_primary = ''
    for aa in partial_primary:
        if aa not in valid_aa:
            new_partial_primary += '*'
        else:
            new_partial_primary += aa
    alignment = global_pairwise_align_protein(Protein(new_primary), Protein(new_partial_primary))
    aligned_primary = str(alignment[0][1]).rstrip()
    result = []
    i = 0
    dssp_structures = ['H', 'B', 'E', 'G', 'I', 'T', 'S']
    for aa in aligned_primary:
        if i >= len(secondary):
            result.append('C')
            continue
        if aa == '-':
            result.append('C')
        elif secondary[i] in dssp_structures:
            result.append(secondary[i])
            i += 1
        else:
            result.append('C')
            i += 1
    return ''.join(result), str(alignment[0][1]).rstrip()

In [7]:
with open('data/casp11/training_100') as raw_data:
    lines = []
    count = 0
    for line in raw_data:
        lines.append(line)
        if len(lines) == 33:
            example = make_training_example(lines)
            lines = []
            partial_primary, secondary = get_secondary(example['id'][:4].lower() + '.dssp')
            if len(partial_primary) > len(example['primary']):
                continue
            else:
                count += 1
            aligned_secondary, aligned_primary = align_secondary(example['primary'].rstrip(), partial_primary.rstrip(), secondary.rstrip())
            example['secondary'] = aligned_secondary
            print(example['primary'] + aligned_primary + '\n' + example['secondary'] + '\n')
            with open('data/processed/train_100', 'a') as out_file:
                out_file.write(example['id'] + '\n')
                out_file.write('\n')
                out_file.write(example['primary'] + '\n')
                out_file.write('\n')
                out_file.write(example['pssm'] + '\n')
                out_file.write('\n')
                out_file.write(example['secondary'] + '\n')
                out_file.write('\n')
                out_file.write(example['tertiary'] + '\n')
                out_file.write('\n')
            if count > 5:
                break

/home/nishaq/Documents/research/PTSP/.ptsp/lib/python3.6/site-packages/skbio/alignment/_pairwise.py:599: EfficiencyWarning: You're using skbio's python implementation of Needleman-Wunsch alignment. This is known to be very slow (e.g., thousands of times slower than a native C implementation). We'll be adding a faster version soon (see https://github.com/biocore/scikit-bio/issues/254 to track progress on this).
  "to track progress on this).", EfficiencyWarning)


GSNVFNNTITHPNAGPTSATSTSTSSNGNTPLSSNSSMNPKSLTDPKLLKNIPMWLKSLRLHKYSDALSGTPWIELIYLDDETLEKKGVLALGARRKLLKAFGIVIDYKERDLIDRSAY
--------------------------------------NPKSLTDPKLLKNIPMWLKSLRLHKYSDALSGTPWIELIYLDDETLEKKGVLALGARRKLLKAFGIVIDYKERDLIDRSAY
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCHHHHSCHHHHHCHHHHHHHHTCHHHHHHHTTSCHHHHTTCCHHHHHHHSCCCHHHHHHHHHHHHHHHHHHHHTCSCGGGC

SLSSTELGDLFWSWLRDGLREGDIPVNTADACVHLTCGFVFISVPGVFFLFLKSHSRSCSSGLKESGRKEQVQAAFEKMRKHRVSDSRRFWQCCLYEEPGGRGRYKKLTGYLIKMSEIYANGNFPDDSLFLKVIN
-LSSTELGDLFWSWLRDGLREGDIPVNTADACVHLTCGFVFISVPGVFFLFLKSH-----------GRKEQVQAAFEKXRKHRVSDSRRFWQCCLYEEPGGRGRYKKLTGYLIKXSEIY-NGNFPDDSLFLKVI-
CCCHHHHHHHHHHHHHHHHHTTCSCBSSTTCSEEEETTEEEEETTHHHHHHHHHCCCCCCCCCCCCCHHHHHHHHHHTTCCSCCBTTBCCEEEEEEEETTTEEEEEEEEEEEEEHHHHCCCCCCCCCCSSEEECC

MAHSSATAGPQADYSGEIAELYDLVHQGKGKDYHREAADLAALVRRHSPKAASLLDVACGTGMHLRHLADSFGTVEGLELSADMLAIARRRNPDAVLHHGDMRDFSLGRRFSAVTCMFSSIGHLAGQAELDAALERFAAHVLPDGVVVVEPWWFPENFTPGYVAAGTVEAGGTTVTRVSHSSREGEATRIEVHYLVAGPDRGITHHEESHRITLFTREQYERAFTAAGLS